<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/netCDF4_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script um die netCDF4 Vorhersagewerte in einen Dataframe abzuspeichern

## Allgemeine Einstelllungen:

In [1]:
!pip install netCDF4

In [2]:
from google.colab import drive

# Mounten des Google-Drive Kontos
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import der benötigten Module
import os
import netCDF4 as nc
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [4]:
# Zeit wird in Stunden seit 1.1.1900 00:00 angegeben
origin = dt.datetime(1900, 1, 1, 0, 0, 0, 0)

In [5]:
# Drive Ordner mit den Vorhersagewerten im NetCDF4-Format
drive_folder = '/content/drive/My Drive/Colab_Notebooks/netCDF4_Wetterdaten/'

## Einlesen der NetCDF4-Dateien:

In [6]:
def read_nc_files_in_google_drive(folder_path):
    nc_files = []  # Liste, um die NetCDF4-Datensätze abzuspeichern

    if os.path.exists(folder_path):
        # Durchsuchen des Drive-Ordners nach Dateien
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            # Überprüfen, ob die Datei eine NetCDF4-Datei ist
            if file_name.endswith(".nc"):
                try:
                    # Öffnen der NetCDF4-Datei und huzfügen zur Liste
                    nc_file = nc.Dataset(file_path)
                    nc_files.append(nc_file)
                except Exception as e:
                    print(f"Fehler beim Öffnen von {file_name}: {str(e)}")

    return nc_files

In [7]:
# Funktionsaufruf zum Einlesen der NetCDF4-Dateien
netcdf_files = read_nc_files_in_google_drive(drive_folder)

In [8]:
# Variablen Deklaration
latitudes = np.empty((1, 1))
longitudes = np.empty((1, 1))
time = np.empty((1, 1))
aod469 = np.empty((1, 1))
aod550 = np.empty((1, 1))
aod670 = np.empty((1, 1))
aod865 = np.empty((1, 1))
gtco3 = np.empty((1, 1))
uvbed = np.empty((1, 1))
uvbedcs = np.empty((1, 1))
hcc = np.empty((1, 1))
lcc = np.empty((1, 1))
mcc = np.empty((1, 1))
tcc = np.empty((1, 1))
cbh = np.empty((1, 1))

In [9]:
# Zuordnen der Variabeln zu entsprechenden NP-Arrays
# fehlende Werte werden durch NaN Werte ersetzt, das mehrdimensionale Array wird zu einem eindimensonalen formatiert
for nc_file in netcdf_files:
    latitudes = np.append(latitudes, nc_file.variables['latitude'][0])
    longitudes = np.append(longitudes, nc_file.variables['longitude'][0])
    time = np.append(time, nc_file.variables['time'][:].filled(np.nan))
    aod469 = np.append(aod469, nc_file.variables['aod469'][:].filled(np.nan).reshape(-3))
    aod550 = np.append(aod550, nc_file.variables['aod550'][:].filled(np.nan).reshape(-3))
    aod670 = np.append(aod670, nc_file.variables['aod670'][:].filled(np.nan).reshape(-3))
    aod865 = np.append(aod865, nc_file.variables['aod865'][:].filled(np.nan).reshape(-3))
    gtco3 = np.append(gtco3, nc_file.variables['gtco3'][:].filled(np.nan).reshape(-3))
    uvbed = np.append(uvbed, nc_file.variables['uvbed'][:].filled(np.nan).reshape(-3))
    uvbedcs = np.append(uvbedcs, nc_file.variables['uvbedcs'][:].filled(np.nan).reshape(-3))
    hcc = np.append(hcc, nc_file.variables['hcc'][:].filled(np.nan).reshape(-3))
    lcc = np.append(lcc, nc_file.variables['lcc'][:].filled(np.nan).reshape(-3))
    mcc = np.append(mcc, nc_file.variables['mcc'][:].filled(np.nan).reshape(-3))
    tcc = np.append(tcc, nc_file.variables['tcc'][:].filled(np.nan).reshape(-3))
    cbh = np.append(cbh, nc_file.variables['cbh'][:].filled(np.nan).reshape(-3))

In [10]:
print(len(time), len(aod469), len(aod550), len(aod670), len(aod865), len(aod865), len(uvbed), len(uvbedcs), len(hcc), len(lcc), len(mcc), len(tcc), len(cbh))

11689 11689 11689 11689 11689 11689 11689 11689 11689 11689 11689 11689 11689


In [11]:
# richtige DateTime berechen
datetime = np.empty((1,1))
datetime = np.array([origin + dt.timedelta(hours=int(i)) for i in time])

In [12]:
# Zuorden der NP-Arrays in einen Dataframe
df_cams = pd.DataFrame({'time': datetime, 'aod469': aod469, 'aod550': aod550,'aod670': aod670,'aod865': aod865,'aod865': gtco3,
                        'uvbed': uvbed,'uvbedcs': uvbedcs, 'hcc': hcc, 'lcc': lcc, 'mcc': mcc, 'tcc': tcc, 'cbh': cbh})

In [13]:
# Index des DF setzen
df_cams.set_index('time', inplace=True)

In [14]:
# Sontieren des DataFrame in die richtige Reihenfolge
df_cams_sorted = pd.DataFrame()
df_cams_sorted = df_cams.sort_index().drop(df_cams.index[0])

In [ ]:
# Ausgabe des sotierten Dataframes
df_cams_sorted

In [16]:
# Prüfen, ob der Dataframe alle einträge enthält
for i in range(len(df_cams_sorted) - 1):
    time_diff = df_cams_sorted.index[i + 1] - df_cams_sorted.index[i]
    if time_diff != pd.to_timedelta('1H'):
        print(f"Index {i} und Index {i + 1} haben keinen Abstand von einer Stunde.")

## Abspeichern der Vorhersagewerte mit stündlicher Auflösung:

In [22]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstellen des vollen Pfades zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichern des DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [ ]:
# Abspeichern des konvertierten DF als CSV
dateiname = 'CSV_Cams_std'
save_dataframe_to_drive(df_cams_sorted, '/content/drive/My Drive/Colab_Notebooks/CSV_Vorhersage', dateiname)

## Abspeichern des Dataframe mit 2 Min. Auflösung.

In [17]:
# Resampling der Zeitreihe auf 2 Min. und interpolation der fehlenden Messwerte
df_cams_resampled = df_cams.resample('2T')
df_cams_interpolated = df_cams_resampled.interpolate(method='polynomial', order = 1)

In [ ]:
# Abspeichern des konvertierten DF als CSV
dateiname = 'CSV_Cams_2M'
save_dataframe_to_drive(df_cams_interpolated, '/content/drive/My Drive/Colab_Notebooks/CSV_Vorhersage', dateiname)